In [111]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import datetime as datetime

In [112]:
import re

def mark(soup):
    td_tags = soup.find_all('td', {'data-th' : "Mark"})
    mark = []
    for tag in td_tags:
        mark.append(tag.text.strip())      
    return mark

def competitor(soup):
    comp= soup.find_all('td',{'data-th':"Competitor"} )
    comps = []
    for tags in comp:
        comps.append(tags.text.strip())
    return comps

def nat(soup): 
    nat= soup.find_all('td',{'data-th' : "Nat"} )
    nats = []
    for tags in nat:
        nats.append(tags.text.strip())
    return nats

def date(soup): # Gets the date the competition is held
    date = soup.find_all('td', {'data-th' : "Date"})
    dates = []
    for tags in date:
        dates.append(tags.text.strip())
    return dates



def page_scrape(page):
    html = requests.get(f'https://worldathletics.org/records/all-time-toplists/middlelong/1500-metres/all/men/senior?regionType=world&page={page}&bestResultsOnly=true&ageCategory=senior')
    if html.status_code != 200: 
        raise Exception ('error {}'.format(html))
    soup = bs(html.text)
    athletics_dict = {
    'Mark':mark(soup),
    'Competitor':competitor(soup),
    'country':nat(soup),
    'date':date(soup),
    }        
    return pd.DataFrame(athletics_dict)

In [113]:
df = page_scrape(1)
for page in range(2, 50):
    df = pd.concat([df, page_scrape(page)])


In [114]:
def to_seconds(time_string):
    min = float(time_string[0:1])
    sec = float(time_string[2:4])
    if "h" in time_string:
        millisec = float(time_string[-3:-1])
    else:
        millisec = float(time_string[-3:])
    return 60 * min + sec + millisec

df['Mark'] = df['Mark'].apply(to_seconds)
df['date'] = pd.to_datetime(df['date'], format='%d %b %Y')

In [115]:
sorted_df = df.sort_values(by='date', ascending=True)

In [116]:
best_time_df = pd.DataFrame()
best_time = 10000

for idx, row in sorted_df.iterrows():
    if best_time >= row["Mark"]:
        best_time = row["Mark"]
        best_time_df = pd.concat([best_time_df, row.to_frame().T])  # Correct concatenation


In [117]:
best_time_df

,Mark,Competitor,country,date
50,223.0,Gunder HÄGG,SWE,1944-07-07 00:00:00
51,223.0,Lennart STRAND,SWE,1947-07-15 00:00:00
52,223.0,Werner LUEG,FRG,1952-06-29 00:00:00
11,222.8,Wes SANTEE,USA,1954-06-04 00:00:00
63,221.8,John LANDY,AUS,1954-06-21 00:00:00
47,220.8,Sándor IHAROS,HUN,1955-07-28 00:00:00
48,220.8,László TÁBORI,HUN,1955-09-06 00:00:00
49,220.8,Gunnar NIELSEN,DEN,1955-09-06 00:00:00
61,220.3,Olavi VUORISALO,FIN,1957-07-11 00:00:00
75,220.2,Olavi SALSOLA,FIN,1957-07-11 00:00:00
